###### Specifing run configurations and regularization constants. These are used to tweak the model.

In [1]:
# RUN CONFIGURATIONS
TEST = 10
CNN_ACT_F = "relu"
DNS_ACT_F = "tanh"
ARCH = f"CNN(64,128,256)_{CNN_ACT_F}_DENSE(128,64,32,12)_{DNS_ACT_F}"
IMG_SIZE = 50
N_SPLIT = 5
EPOCHS = 30
BATCH_SIZE = 50
VAL_SPLIT = 0.1


# REGULARIZATIONS
DROPOUT_RATE = 0.3
LAMBDA_1 = 1e-6
LAMBDA_2 = 1e-6

###### Loading the dataset from pickle

In [2]:
import pickle

X = pickle.load(open(f"X_{IMG_SIZE}.pickle", "rb"))
Y = pickle.load(open(f"Y_{IMG_SIZE}.pickle", "rb"))

###### Creating the report name for the corresponding tweak of the model

In [3]:
REPORT_NAME = f"{TEST} - {ARCH} - {IMG_SIZE}px_{EPOCHS}e_{BATCH_SIZE}bs_{int(VAL_SPLIT*100)}vs_REG_{int(DROPOUT_RATE*100)}do_{int(LAMBDA_1*1000000)}L1_{int(LAMBDA_2*1000000)}L2"
REPORT_NAME

'10 - CNN(64,128,256)_relu_DENSE(128,64,32,12)_tanh - 50px_30e_50bs_10vs_REG_30do_1L1_1L2'

###### Importing tensorflow-gpu and other related libs and classes. Tensorboard is used to see live tables.

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import time

C:\Users\Thilina\.conda\envs\tensorflow-gpu-1-13\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Thilina\.conda\envs\tensorflow-gpu-1-13\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Thilina\.conda\envs\tensorflow-gpu-1-13\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Thilina\.conda\envs\tenso

In [5]:
MODEL_NAME = f"NN_GA_CNN_{int(time.time())}"

tensorboard = TensorBoard(log_dir=f"logs/{MODEL_NAME}")

def create_cnn_model():
    model = Sequential()
    
    model.add(Conv2D(64, (3,3), input_shape=(IMG_SIZE,IMG_SIZE,1)))  # input shape is IMG_SIZExIMG_SIZEx1
#     model.add(BatchNormalization(axis=3))
    model.add(Activation(CNN_ACT_F))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
    model.add(Conv2D(128, (3,3)))
    model.add(Activation(CNN_ACT_F))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
    model.add(Conv2D(256, (3,3)))
    model.add(Activation(CNN_ACT_F))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(128, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Dropout(rate=DROPOUT_RATE))
    
    model.add(Flatten())
    
#     model.add(Dense(128))
    model.add(Dense(128, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
    model.add(Activation(DNS_ACT_F))
    
    model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
    model.add(Activation(DNS_ACT_F))
    
    model.add(Dense(32, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
    model.add(Activation(DNS_ACT_F))
    
    model.add(Dense(12))
    model.add(Activation("softmax"))
    
    optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

###### Creating folder for saving reports of the model executions

In [6]:
import os

if not os.path.exists('reports'):
    os.makedirs('reports')

file = open(f"reports/{REPORT_NAME}.txt", "w")

###### Designed the 5-fold cross validation training and test with specific validation split. Final accuracy is averaged by the accuracies of the separate 5 folds. Time taken for the execution is also measured.

In [7]:
from sklearn.model_selection import KFold

# initializing variables for kFold run and average accuracy
current_fold = 0
sum_acc = 0
avg_acc = 0

start = time.process_time()

for train_index, test_index in KFold(N_SPLIT).split(X):
    current_fold += 1
    
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    model = create_cnn_model()
    
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VAL_SPLIT, callbacks=[tensorboard])
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(f"\nFOLD-{current_fold}: Loss={val_loss} , Accuracy={val_acc}\n")
    
    sum_acc += val_acc 
    file.write(f"{current_fold}-FOLD | Loss={round(val_loss,4)},\tAccuracy={round(val_acc,4)},\tAverage_Accuracy={round(sum_acc/current_fold,4)}\n")
    
    if(current_fold == N_SPLIT):
        avg_acc = round(sum_acc/current_fold,4)
    

avg_acc_line = f"\nAverage Accuracy : {round(avg_acc,4)}"

end = time.process_time()
time_taken = f"\nExecution Time\t : {round(end-start,4)}s"

print(avg_acc_line)
print(time_taken)

file.write(avg_acc_line)
file.write(time_taken)
    
file.close()

Instructions for updating:
Colocations handled automatically by placer.
Train on 3420 samples, validate on 380 samples
Epoch 1/30
3420/3420 [==============================] - 8s 2ms/sample - loss: 2.4312 - acc: 0.1357 - val_loss: 2.4798 - val_acc: 0.1158
Epoch 2/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.3977 - acc: 0.1523 - val_loss: 2.4466 - val_acc: 0.1711
Epoch 3/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.3770 - acc: 0.1599 - val_loss: 2.4393 - val_acc: 0.1500
Epoch 4/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.3727 - acc: 0.1564 - val_loss: 2.4541 - val_acc: 0.1447
Epoch 5/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.3541 - acc: 0.1614 - val_loss: 2.4269 - val_acc: 0.1553
Epoch 6/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.3408 - acc: 0.1667 - val_loss: 2.4122 - val_acc: 0.1553
Epoch 7/30
3420/3420 [==============================] - 4s 1ms/sa

Epoch 27/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 0.7945 - acc: 0.7357 - val_loss: 1.4202 - val_acc: 0.5289
Epoch 28/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 0.7279 - acc: 0.7637 - val_loss: 1.3939 - val_acc: 0.5447
Epoch 29/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 0.6062 - acc: 0.8003 - val_loss: 1.4731 - val_acc: 0.5474
Epoch 30/30
950/950 [==============================] - 1s 538us/sample - loss: 1.3426 - acc: 0.5811

FOLD-2: Loss=1.3426102483899969 , Accuracy=0.5810526609420776

Train on 3420 samples, validate on 380 samples
Epoch 1/30
3420/3420 [==============================] - 5s 2ms/sample - loss: 2.4398 - acc: 0.1266 - val_loss: 2.4614 - val_acc: 0.1447
Epoch 2/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.4127 - acc: 0.1433 - val_loss: 2.4579 - val_acc: 0.1211
Epoch 3/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.3930 - acc: 0.1515 - val_l

3420/3420 [==============================] - 4s 1ms/sample - loss: 0.9353 - acc: 0.6863 - val_loss: 2.0778 - val_acc: 0.3947
Epoch 25/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 0.8738 - acc: 0.7050 - val_loss: 1.5388 - val_acc: 0.5263
Epoch 26/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 0.8105 - acc: 0.7310 - val_loss: 1.5672 - val_acc: 0.5079
Epoch 27/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 0.7307 - acc: 0.7591 - val_loss: 1.4864 - val_acc: 0.5316
Epoch 28/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 0.6049 - acc: 0.8058 - val_loss: 1.5674 - val_acc: 0.5316
Epoch 29/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 0.4900 - acc: 0.8541 - val_loss: 1.5709 - val_acc: 0.5868
Epoch 30/30
950/950 [==============================] - 1s 593us/sample - loss: 1.5178 - acc: 0.5358

FOLD-4: Loss=1.517762599493328 , Accuracy=0.5357894897460938

Train on 3420 samples, vali

###### Saving the model for later use

In [8]:
model.save(f"{REPORT_NAME}.model")

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 48, 48, 64)        640       
_________________________________________________________________
activation_28 (Activation)   (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 21, 21, 128)       73856     
_________________________________________________________________
activation_29 (Activation)   (None, 21, 21, 128)       0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 8, 256)         295168    
__________

###### To reset GPU memory after model training. But seems this doesn't work. Always need to restart the kernel and run.

In [10]:
tf.reset_default_graph()